In [ ]:
import sys
import pathlib
#sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))
#sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().joinpath('Quantum_Annealing_for_Particle_Matching'))))
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))
for p in sys.path:
    print(p)
import os
import time
import h5py
import cupy as np
#import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import dwave
import dwave.system
import src
from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import mixing_models, io
import src.leap_funcs.qubo.q_matrix as q_matrix
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import filter_samples
from src._misc import compare_matrices

from src import h5py_funcs
from src.h5py_funcs import discoveries, init_custom_getstates, io, parameterstudy_using_info_file

In [ ]:
b = np.zeros((12000, 12000), dtype=np.float16)
print(b.nbytes/10e9)
print(b.dtype)
print((12000**2)*2*10e-9)

In [ ]:
sim_path = pathlib.Path(r'C:\\Users\\oahre\\RWTH\\OneDrive_Students_RWTH\\RWTH\Simulation_Sciences\\HiWi\\Rocky\\99_Paper_Clemens_Performance\\11_Particles_17332_CPU_16_GPU_0.rocky.files')
#sim_path = pathlib.Path(r'C:\\Users\\oahre\\RWTH\\OneDrive_Students_RWTH\\RWTH\Simulation_Sciences\\HiWi\\Rocky\\99_Paper_Clemens_Performance\\14_Particles_36308_CPU_16_GPU_0.rocky.files')
dem_data_dict = src.particle_funcs.io.read_dem_data(sim_path)

In [ ]:
print(dem_data_dict['rocky_simulation00000.rhs']['Particles']['particles_position']['data'].view((np.float64,3)).copy())
part_pos_array = np.array(dem_data_dict['rocky_simulation00000.rhs']['Particles']['particles_position']['data'].view((np.float64,3)))
part_pos_array = np.array(part_pos_array)
part_mat_id = np.zeros(part_pos_array.shape[0])
part_mat_id[np.where(part_pos_array[:,0]>0.0)] = 1
part_mat_id

In [ ]:
file_list = list(dem_data_dict.keys())
file_list.remove('num_particles')
file_list.remove('particle_index')
num_timesteps = len(file_list)
num_particles = part_pos_array.shape[0]

In [ ]:
datatype = np.float16
part_pos_array_all = np.zeros((num_timesteps,num_particles,3), dtype=datatype)
for i,f in enumerate(file_list):
    for j,a in enumerate(['x','y','z']):
            part_pos_array_all[i,:,j] = np.array(dem_data_dict[f]['Particles']['particles_position']['data'][a].astype(datatype))

In [ ]:
mixing_models_names = ['average_height', 'nearest_neighbour', 'lacey_index', 'mixing_entropy', 'particle_scale_index']
mixing_models_dict = {}

In [ ]:
print(part_pos_array_all.dtype)
distance_matrix.calc_phi_ij(part_pos_array_all[0],part_pos_array_all[0])

In [ ]:
for t in range(1):
    print(mixing_models.average_height(coords=part_pos_array_all[t], axis=0, species=part_mat_id, normalize=0, verbose=0))
    #print(mixing_models.nearest_neighbour(coords=part_pos_array_all[t], species=part_mat_id, num_neighbours=12, kwargs_distance_matrix={}, verbose=0))
    print(mixing_models.calc_lacey_index(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0))
    print(mixing_models.calc_mixing_entropy(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, method='combined', kwargs_distance_matrix={}, verbose=0))
    print(mixing_models.calc_coordination_number_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, kwargs_distance_matrix={}, verbose=0, _internal=0))
    print(mixing_models.calc_particle_scale_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0))

In [ ]:
mixing_models.average_height(coords=part_pos_array_all[t], axis=0, species=part_mat_id, normalize=0, verbose=0)
mixing_models.nearest_neighbour(coords=part_pos_array_all[t], species=part_mat_id, num_neighbours=12, kwargs_distance_matrix={}, verbose=0)
mixing_models.calc_lacey_index(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0)
mixing_models.calc_mixing_entropy(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, method='combined', kwargs_distance_matrix={}, verbose=0)
mixing_models.calc_coordination_number_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, kwargs_distance_matrix={}, verbose=0, _internal=0)
mixing_models.calc_particle_scale_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0)